# Pre-requisite installations if needed

In [8]:
!pip install oauth2client
!pip install google-api-python-client
!pip install ipywidgets
!pip install plotly
!pip install tqdm


SyntaxError: invalid syntax (<ipython-input-8-fc27e90ce6c5>, line 7)

# ------------

In [12]:
import argparse
import pandas as pd
import json
import os
import ipywidgets as widgets
from pathlib import Path
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from tqdm.notebook import tqdm, trange
import time

import plotly
import plotly.graph_objs as go

from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

import httplib2
from oauth2client import client
from oauth2client import file
from oauth2client import tools
from helper_functions import initialize_analyticsreporting, get_report, print_response, VIEW_ID

analytics = initialize_analyticsreporting()
# from plotly.graph_objs import Scatter, Layout

# plotly.offline.plot({
#     "data": [Scatter(x=[1, 2, 3, 4], y=[4, 3, 2, 1])],
#     "layout": Layout(title="hello world")
# })

In [13]:
for i in trange(100):
    time.sleep(0.01)

  0%|          | 0/100 [00:00<?, ?it/s]

In [2]:
feature = widgets.Dropdown(
    options=['App Launched - OS', '2', '3'],
    value='App Launched - OS',
    description='Option:',
    disabled=False,
)

start_date = widgets.DatePicker(description='Start Date:', disabled=False)
end_date = widgets.DatePicker(description='End Date:', disabled=False)

update_interval = widgets.Dropdown(options=['Daily', 'Weekly', 'Monthly', 'Yearly'], description='Update Interval:', disabled=False)

display(feature, start_date, end_date, update_interval)

Dropdown(description='Option:', options=('App Launched - OS', '2', '3'), value='App Launched - OS')

DatePicker(value=None, description='Start Date:')

DatePicker(value=None, description='End Date:')

Dropdown(description='Update Interval:', options=('Daily', 'Weekly', 'Monthly', 'Yearly'), value='Daily')

In [16]:
if update_interval.value == "Daily":
    dt = start_date.value
    ds = end_date.value
    start = dt
    end = start
    
    data = []
       
    for i in trange((ds - dt).days):
        if start <= ds:
            query = {
                'reportRequests': [
                {
                    'viewId': VIEW_ID,
                    'dateRanges': [{'startDate': start.strftime('%Y-%m-%d'), 'endDate': end.strftime('%Y-%m-%d')}],
                    'metrics': [{'expression': 'ga:totalEvents'}],
                    'dimensions': [{'name': 'ga:eventAction'}]
                }]
            }
            response = get_report(analytics, query)
            if "rows" in response["reports"][0]["data"]:
                response_rows = response["reports"][0]["data"]["rows"]
            else:
                response_rows = []
                cell_data = [ start.strftime("%d %b, %Y") , 0]
                data.append(cell_data)

            for res in response_rows:
                if res["dimensions"][0] == feature.value:
                    if response_rows != []:
                        cell_data = [ start.strftime("%d %b, %Y"), res["metrics"][0]["values"][0]]
                        data.append(cell_data)
                    else:
                        cell_data = [ start.strftime("%d %b, %Y") , 0]
                        data.append(cell_data)
                    
            start = start + timedelta(days=1)
            end = start 
    
    folder_path = os.path.join("result_csv", "graph_data")
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame(data, columns = ['Date Interval', 'Frequency'])
    result_path = os.path.join(folder_path, "daily_graph-" + dt.strftime("%d %b, %Y") + "-" + ds.strftime("%d %b, %Y") + ".csv")
    df.to_csv(result_path, encoding='utf-8', index=False)
    
    action_column = df.iloc[:, 0]
    frequency_column = df.iloc[:, 1]
    x_markers = pd.Series(action_column).array
    y_markers = pd.Series(frequency_column).array
    y_markers = y_markers.astype(int)
    
    print(x_markers, y_markers)
    trace0 = go.Scatter(
        x = x_markers,
        y = y_markers
    )
    
    plotly.offline.plot([trace0])


  0%|          | 0/29 [00:00<?, ?it/s]

In [8]:
dt = start_date.value
ds = end_date.value
start = dt - timedelta(days=dt.weekday())
end = start + timedelta(days=6)
while start <= ds:
    print(start, end)
    start = start + timedelta(days=7)
    end = start + timedelta(days=6)

2021-04-19 2021-04-25
2021-04-26 2021-05-02
2021-05-03 2021-05-09
2021-05-10 2021-05-16
2021-05-17 2021-05-23
2021-05-24 2021-05-30
2021-05-31 2021-06-06
2021-06-07 2021-06-13
2021-06-14 2021-06-20
2021-06-21 2021-06-27


In [8]:
# Date format in 'YYYY-MM-DD'
# You can also use relative dates for simplicity
# start_date = "50daysAgo"
# end_date = "today"
# end_date = "yesterday"
start_date = "2021-01-23"
end_date = "2021-04-23"

### Get a report of all the events that occured with a status of either "success" or "error" within a given time frame

In [10]:
query = {
    'reportRequests': [
    {
        'viewId': VIEW_ID,
        'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
        'metrics': [{'expression': 'ga:totalEvents'}],
        'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}]
    }]
}

response = get_report(analytics, query)
response_rows = response["reports"][0]["data"]["rows"]
both_data, success_data, error_data = [], [], []

for res in response_rows:
    both_cell_data = [res["dimensions"][0], res["dimensions"][1], res["metrics"][0]["values"][0]]
    both_data.append(both_cell_data)
    if res["dimensions"][0] == "Success":
        success_cell_data = [res["dimensions"][1], res["metrics"][0]["values"][0]]
        success_data.append(success_cell_data)
    if res["dimensions"][0] == "Error":
        error_cell_data = [res["dimensions"][1], res["metrics"][0]["values"][0]]
        error_data.append(error_cell_data)

folder_path = os.path.join("result_csv", "status_count")
Path(folder_path).mkdir(parents=True, exist_ok=True)
        
df = pd.DataFrame(both_data, columns = ['Status', 'Action', 'Values'])
result_path = os.path.join(folder_path, "Both-" + start_date + "_" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

df = pd.DataFrame(success_data, columns = ['Action', 'Values'])
result_path = os.path.join(folder_path, "Success-" + start_date + "_" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

df = pd.DataFrame(error_data, columns = ['Action', 'Values'])
result_path = os.path.join(folder_path, "Error-" + start_date + "_" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

### Get a report of all app launches within a given time frame

In [11]:
query = {
    'reportRequests': [
    {
        'viewId': VIEW_ID,
        'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
        'metrics': [{'expression': 'ga:totalEvents'}],
        'dimensions': [{'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
    }]
}

response = get_report(analytics, query)
response_rows = response["reports"][0]["data"]["rows"]
app_launch_os, app_launch_soda = [], []

for res in response_rows:
    if res["dimensions"][0] == "App Launched - OS":
        app_launch_os_cell_data = [res["dimensions"][1], res["metrics"][0]["values"][0]]
        app_launch_os.append(app_launch_os_cell_data)
    if res["dimensions"][0] == "App Launched - SODA":
        app_launch_soda_cell_data = [res["dimensions"][1], res["metrics"][0]["values"][0]]
        app_launch_soda.append(app_launch_soda_cell_data)

folder_path = os.path.join("result_csv", "app_launched")
Path(folder_path).mkdir(parents=True, exist_ok=True)
        
df = pd.DataFrame(app_launch_os, columns = ['OS', 'Values'])
result_path = os.path.join(folder_path, "os-" + start_date + "_" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

df = pd.DataFrame(app_launch_soda, columns = ['SODA Version', 'Values'])
result_path = os.path.join(folder_path, "soda_version-" + start_date + "_" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

### Get a report of all unique users within a given time frame

In [12]:
query = {
    'reportRequests': [
    {
        'viewId': VIEW_ID,
        'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
        'metrics': [{'expression': 'ga:users'}],
        'dimensions': [{'name': 'ga:userType'}]
    }]
}

response = get_report(analytics, query)
response_rows = response["reports"][0]["data"]["rows"]
data = []

for res in response_rows:
    cell_data = [res["dimensions"][0], res["metrics"][0]["values"][0]]
    data.append(cell_data)
    
folder_path = os.path.join("result_csv", "users")
Path(folder_path).mkdir(parents=True, exist_ok=True)
        
df = pd.DataFrame(data, columns = ['Type', 'Values'])
result_path = os.path.join(folder_path, "users-" + start_date + "_" + end_date + ".csv")
df.to_csv(result_path, encoding='utf-8', index=False)

### Get a report of all dataset names for a specific action for a given time frame

In [13]:
def number_of_actions(start_date, end_date, category, action):
    query = {
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': [{'expression': 'ga:totalEvents'}],
            'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
        }]
    }
    response = get_report(analytics, query)
    response_rows = response["reports"][0]["data"]["rows"]
    data = []

    for res in response_rows:
        if res["dimensions"][0] == category:
            if res["dimensions"][1] == action:
                cell_data = [res["dimensions"][2], res["metrics"][0]["values"][0]]
                data.append(cell_data)
        
    folder_path = os.path.join("result_csv", "custom")
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame(data, columns = ['Dataset_name', 'Values'])
    result_path = os.path.join(folder_path, action + "-" + start_date + "_" + end_date + ".csv")
    df.to_csv(result_path, encoding='utf-8', index=False)
    return

## useful for getting the names of datasets where an action is applicable
## all responses go to the custom folder
# number_of_actions(start_date, end_date, <type>, <action_name>)
number_of_actions(start_date, end_date, "Success", "Manage Dataset - Create Empty Dataset")

### Get a report of all actions done on a specific dataset for a given time frame

In [14]:
def dataset_actions(start_date, end_date, dataset_name):
    query = {
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': [{'expression': 'ga:totalEvents'}],
            'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
        }]
    }
    response = get_report(analytics, query)
    response_rows = response["reports"][0]["data"]["rows"]
    data = []

    for res in response_rows:
        if res["dimensions"][2].find(dataset_name) != -1 or res["dimensions"][1].find(dataset_name) != -1:
            cell_data = [res["dimensions"][0], res["dimensions"][1], res["metrics"][0]["values"][0]]
            data.append(cell_data)
        
    folder_path = os.path.join("result_csv", "custom")
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame(data, columns = ['Status', 'Action', 'Values'])
    result_path = os.path.join(folder_path, dataset_name + "(actions)-" + start_date + "_" + end_date + ".csv")
    df.to_csv(result_path, encoding='utf-8', index=False)
    return

## useful for getting all the actions for a specific dataset
## all responses go to the custom folder
# dataset_actions(start_date, end_date, <Dataset_name>)
dataset_actions(start_date, end_date, "test-ps-SODA")

### Get the number of files and the size of all datasets that was uploaded through SODA for a given time frame

In [15]:
def dataset_statistics(start_date, end_date):
            
    query = {
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': [{'expression': 'ga:uniqueEvents'}],
            'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
        }]
    }
    response = get_report(analytics, query)
    response_rows = response["reports"][0]["data"]["rows"]
    
    for res in response_rows:
#         print(res)
        if res["dimensions"][1] == "Upload Local Dataset - Number of Files":
            cell_data = [res["dimensions"][0], res["dimensions"][2], 0]
            data.append(cell_data)
        if res["dimensions"][1] == "Upload Local Dataset - size":
            cell_data = [res["dimensions"][0], 0, res["dimensions"][2]]
            data.append(cell_data)
        if res["dimensions"][1] == "Generate Dataset - Number of Files":
            cell_data = [res["dimensions"][0], res["dimensions"][2], 0]
            data.append(cell_data)
        if res["dimensions"][1] == "Generate Dataset - Size":
            cell_data = [res["dimensions"][0], 0, res["dimensions"][2]]
            data.append(cell_data)
        
    folder_path = os.path.join("result_csv", "custom")
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame(data, columns = ['Status', 'Number of Files', 'Size in (bytes)'])
    result_path = os.path.join(folder_path, "dataset_statistics-" + start_date + "_" + end_date + ".csv")
    df.to_csv(result_path, encoding='utf-8', index=False)
    return

## useful for getting all details for upload to Pennsieve for a specific time period
## all responses go to the custom folder
# num_of_files_folders_in_dataset(start_date, end_date)
dataset_statistics(start_date, end_date)

### Get the number of files and the size of a specific dataset that was uploaded through SODA

In [16]:
def num_of_files_folders_in_dataset(start_date, end_date, dataset_name):
    query = {
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': [{'expression': 'ga:totalEvents'}],
            'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
        }]
    }
    response = get_report(analytics, query)
    response_rows = response["reports"][0]["data"]["rows"]
    data = []
    
    for res in response_rows:
        if res["dimensions"][1].find("Upload Local Dataset") != -1:
            if res["dimensions"][1].find(dataset_name) != -1:
                if res["dimensions"][1].find('Number of Files') != -1:
                    cell_data = [res["dimensions"][0], res["dimensions"][2], 0]
                    data.append(cell_data)
                if res["dimensions"][1].find('- size') != -1:
                    cell_data = [res["dimensions"][0], 0, res["dimensions"][2]]
                    data.append(cell_data)
            
    query = {
        'reportRequests': [
        {
            'viewId': VIEW_ID,
            'dateRanges': [{'startDate': start_date, 'endDate': end_date}],
            'metrics': [{'expression': 'ga:eventValue'}],
            'dimensions': [{'name': 'ga:eventCategory'}, {'name': 'ga:eventAction'}, {'name': 'ga:eventLabel'}]
        }]
    }
    response = get_report(analytics, query)
    response_rows = response["reports"][0]["data"]["rows"]
    res["metrics"][0]["values"][0]
    for res in response_rows:
        if res["dimensions"][1].find("Generate Dataset") != -1:
            if res["dimensions"][2].find(dataset_name) != -1:
                if res["dimensions"][1] == 'Generate Dataset - Number of Files':
                    cell_data = [res["dimensions"][0], res["metrics"][0]["values"][0], 0]
                    data.append(cell_data)
                if res["dimensions"][1] == 'Generate Dataset - Size':
                    cell_data = [res["dimensions"][0], 0, res["metrics"][0]["values"][0]]
                    data.append(cell_data)
                
                    
    folder_path = os.path.join("result_csv", "custom")
    Path(folder_path).mkdir(parents=True, exist_ok=True)

    df = pd.DataFrame(data, columns = ['Status', 'Number of Files', 'Size in (bytes)'])
    result_path = os.path.join(folder_path, dataset_name + "(details)-" + start_date + "_" + end_date + ".csv")
    df.to_csv(result_path, encoding='utf-8', index=False)
    return

## useful for getting all the number of files and size for a specific dataset
## all responses go to the custom folder
# num_of_files_folders_in_dataset(start_date, end_date, <Dataset_name>)
num_of_files_folders_in_dataset(start_date, end_date, "test-ps-SODA")